# Copyright Netherlands eScience Center <br>
** Function     : Analyze the trend of net surface flux from coordinated experiments**** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2019.08.09 ** <br>
** Last Update  : 2019.08.29 ** <br>
Description     : This notebook aims to analyze the trend of net surface flux from multiple coordinated experiments in Blue Action WP3. It contributes to the Deliverable 3.1. <br>
Return Values   : netCDF4 <br>
Caveat          : The fields used here are post-processed monthly mean fields. It includes AMET from:
* EC Earth (DMI)
* CMCC-CM (CMCC)
* WACCM6 (WHOI)
* NorESM (NERSC)
* HadGEM  (UoS)
* EC Earth (NLeSC)

In [1]:
%matplotlib inline
import numpy as np
import sys
#sys.path.append("/home/ESLT0068/NLeSC/Computation_Modeling/Bjerknes/Scripts/META")
import analyzer
import visualizer
import scipy as sp
import time as tttt
from netCDF4 import Dataset,num2date
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

In [2]:
# constants
constant = {'g' : 9.80616,      # gravititional acceleration [m / s2]
            'R' : 6371009,      # radius of the earth [m]
            'cp': 1004.64,      # heat capacity of air [J/(Kg*K)]
            'Lv': 2264670,      # Latent heat of vaporization [J/Kg]
            'R_dry' : 286.9,    # gas constant of dry air [J/(kg*K)]
            'R_vap' : 461.5,    # gas constant for water vapour [J/(kg*K)]
            }

|Model and Organization | Exp 1 | Exp 2 | Exp 3 | Exp 4 |    time   |
|-----------------------|-------|-------|-------|-------|-----------|
|    EC Earth (NLeSC)   |   20  |   20  |   20  |   20  | 1979-2015 (exp1&2) 1979-2013 (exp3&4) |
|     EC Earth (DMI)    |   20  |   20  |   20  |   20  | 1979-2015 (exp1&2) 1979-2013 (exp3&4) |
|     CMCC-CM (CMCC)    |   10  |   10  |   0   |   0   | 1979-2014 |
|      WACCM6 (WHOI)    |   30  |   30  |   0   |   0   | 1979-2014 |
|     NorESM (NERSC)    |   20  |   20  |   20  |   20  | 1979-2014 (exp1&2) 1979-2013 (exp3&4) |
|     HadGEM  (UoS)     |   10  |   10  |   5   |   5   | 1979-2014 (exp1&2) 1979-2013 (exp3&4) |
|   IAP-AGCM (IAP-NZC)  |   15  |   15  |   15  |   15  | 1979-2015 |
|     IPSL-CM (CNRS)    |   30  |   30  |   20  |   20  | 1979-2014 |
|      MPIESM (MPI)     |   10  |   10  |   5   |   5   | 1979-2015 |

In [7]:
################################   Input zone  ######################################
# specify starting and ending time
start_year = 1979
end_year = 2015
# specify data path
datapath = '/home/ESLT0068/WorkFlow/Core_Database_BlueAction_WP3/ECEarth_DMI/ENS00_19'
# specify output path for figures
output_path = '/home/ESLT0068/NLeSC/Computation_Modeling/BlueAction/WP3/JointAnalysis_AMET/AMIP/spatial'
# ensemble number
ensemble = 20
# experiment number
exp = 4
# example file
datapath_example = os.path.join(datapath, 'slhf', 'DMI_ecearth3_BA-WP3_AEXP1-ENS1_1979-2015_2D_slhf.nc')
####################################################################################

In [4]:
def var_key_retrieve(datapath, exp_num, ensemble_num):
    # get the path to each datasets
    print ("Start retrieving datasets of experiment {} ensemble number {}".format(exp_num+1, ensemble_num))
    # get data path
    if exp_num<2:
        if ensemble_num<10:    
            datapath_slhf = os.path.join(datapath, 'slhf', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2015_2D_slhf.nc'.format(exp_num+1, ensemble_num))
            datapath_sshf = os.path.join(datapath, 'sshf', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2015_2D_sshf.nc'.format(exp_num+1, ensemble_num))
            datapath_ssr = os.path.join(datapath, 'ssr', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2015_2D_ssr.nc'.format(exp_num+1, ensemble_num))
            datapath_str = os.path.join(datapath, 'str', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2015_2D_str.nc'.format(exp_num+1, ensemble_num))
        else:    
            datapath_slhf = os.path.join(datapath, 'slhf', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2015_2D_slhf.nc4'.format(exp_num+1, ensemble_num))
            datapath_sshf = os.path.join(datapath, 'sshf', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2015_2D_sshf.nc4'.format(exp_num+1, ensemble_num))
            datapath_ssr = os.path.join(datapath, 'ssr', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2015_2D_ssr.nc4'.format(exp_num+1, ensemble_num))
            datapath_str = os.path.join(datapath, 'str', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2015_2D_str.nc4'.format(exp_num+1, ensemble_num))
    else:
        if ensemble_num<10:    
            datapath_slhf = os.path.join(datapath, 'slhf', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2013_2D_slhf.nc'.format(exp_num+1, ensemble_num))
            datapath_sshf = os.path.join(datapath, 'sshf', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2013_2D_sshf.nc'.format(exp_num+1, ensemble_num))
            datapath_ssr = os.path.join(datapath, 'ssr', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2013_2D_ssr.nc'.format(exp_num+1, ensemble_num))
            datapath_str = os.path.join(datapath, 'str', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2013_2D_str.nc'.format(exp_num+1, ensemble_num))
        else:    
            datapath_slhf = os.path.join(datapath, 'slhf', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2013_2D_slhf.nc4'.format(exp_num+1, ensemble_num))
            datapath_sshf = os.path.join(datapath, 'sshf', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2013_2D_sshf.nc4'.format(exp_num+1, ensemble_num))
            datapath_ssr = os.path.join(datapath, 'ssr', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2013_2D_ssr.nc4'.format(exp_num+1, ensemble_num))
            datapath_str = os.path.join(datapath, 'str', 'DMI_ecearth3_BA-WP3_AEXP{}-ENS{}_1979-2013_2D_str.nc4'.format(exp_num+1, ensemble_num))
    # get the variable keys            
    # get the variable keys    
    key_slhf = Dataset(datapath_slhf)
    key_sshf = Dataset(datapath_sshf)
    key_ssr = Dataset(datapath_ssr)
    key_str = Dataset(datapath_str)

    print ("Retrieving datasets successfully and return the variable key!")
    return key_slhf, key_sshf, key_ssr, key_str

In [11]:
def sflux(key_slhf, key_sshf, key_ssr, key_str, lat, lon):
    # get all the varialbes
    # make sure that all the input variables here are positive downward!!!
    var_slhf = key_slhf.variables['slhf'][:,:100,:] # surface latent heat flux W/m2
    var_sshf = key_sshf.variables['sshf'][:,:100,:] # surface sensible heat flux W/m2 
    var_ssr = key_ssr.variables['ssr'][:,:100,:] # surface solar radiation W/m2
    var_str = key_str.variables['str'][:,:100,:] # surface thermal radiation W/m2
    #size of the grid box
    #dx = 2 * np.pi * constant['R'] * np.cos(2 * np.pi * lat /
    #                                        360) / len(lon) 
    #dy = np.pi * constant['R'] / len(lat)
    # calculate total net energy flux at TOA/surface
    net_flux_surf = var_slhf + var_sshf + var_ssr + var_str
    net_flux_surf = net_flux_surf.reshape(-1,12,len(lat),len(lon))

    return net_flux_surf

In [12]:
if __name__=="__main__":
    ####################################################################
    ######  Create time namelist matrix for variable extraction  #######
    ####################################################################
    lat_DMI_20N = 100
    # date and time arrangement
    # namelist of month and days for file manipulation
    namelist_month = ['01','02','03','04','05','06','07','08','09','10','11','12']
    ensemble_list = ['01','02','03','04','05','06','07','08','09','10',
                     '11','12','13','14','15','16','17','18','19','20',
                     '21','22','23','24','25','26','27','28','29','30',]
    # index of months
    period_1979_2015 = np.arange(start_year,end_year+1,1)
    period_1979_2013 = np.arange(start_year,end_year+1-2,1)
    index_month = np.arange(1,13,1)
    ####################################################################
    ######       Extract invariant and calculate constants       #######
    ####################################################################
    # get basic dimensions from sample file
    key_example = Dataset(datapath_example)
    lat = key_example.variables['lat'][:100]
    lon = key_example.variables['lon'][:]
    # get invariant from benchmark file
    Dim_year_1979_2015 = len(period_1979_2015)
    Dim_year_1979_2013 = len(period_1979_2013)
    Dim_month = len(index_month)
    Dim_latitude = len(lat)
    Dim_longitude = len(lon)
    #############################################
    #####   Create space for stroing data   #####
    #############################################
    # data pool
    pool_sflux_1979_2015 = np.zeros((ensemble,Dim_year_1979_2015,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    pool_sflux_1979_2013 = np.zeros((ensemble,Dim_year_1979_2013,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    # loop for calculation
    for i in range(exp):
        for j in range(ensemble):
            # get variable keys
            key_slhf, key_sshf, key_ssr, key_str = var_key_retrieve(datapath, i, j)
            # compute amet
            if i<2:
                pool_sflux_1979_2015[j,:,:,:,:] = sflux(key_slhf, key_sshf, key_ssr,
                                                        key_str, lat, lon)
            else:
                pool_sflux_1979_2013[j,:,:,:,:] = sflux(key_slhf, key_sshf, key_ssr,
                                                        key_str, lat, lon)
        ####################################################################
        ######        Calculating Trend   (positive downward)        #######
        ####################################################################
        if i<2:
            # calculate trend and take ensemble mean
            ens_avg_DMI_exp = analyzer.spatial(np.mean(pool_sflux_1979_2015[:],0))
            ens_avg_DMI_exp.anomaly()
            ens_avg_DMI_exp.trend()
            #ticks = [-0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8]
            ticks = np.linspace(-0.5,0.5,11)
            label = 'Trend of SFlux (W/(m2*decade))'
            visualizer.plots.geograph(lat, lon, ens_avg_DMI_exp._a*10,
                                      label, ticks, os.path.join(output_path,
                                      'trend_spatial_ECEarth_DMI_SFlux_60N_exp_{}.png'.format(i)))
        else:
            ens_avg_DMI_exp = analyzer.spatial(np.mean(pool_sflux_1979_2013[:],0))
            ens_avg_DMI_exp.anomaly()
            ens_avg_DMI_exp.trend()
            #ticks = [-0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8]
            ticks = np.linspace(-0.5,0.5,11)
            label = 'Trend of SFlux (W/(m2*decade))'
            visualizer.plots.geograph(lat, lon, ens_avg_DMI_exp._a*10,
                                      label, ticks, os.path.join(output_path,
                                      'trend_spatial_ECEarth_DMI_SFlux_60N_exp_{}.png'.format(i)))

Start retrieving datasets of experiment 1 ensemble number 0
Retrieving datasets successfully and return the variable key!
Start retrieving datasets of experiment 1 ensemble number 1
Retrieving datasets successfully and return the variable key!
Start retrieving datasets of experiment 1 ensemble number 2
Retrieving datasets successfully and return the variable key!
Start retrieving datasets of experiment 1 ensemble number 3
Retrieving datasets successfully and return the variable key!
Start retrieving datasets of experiment 1 ensemble number 4
Retrieving datasets successfully and return the variable key!
Start retrieving datasets of experiment 1 ensemble number 5
Retrieving datasets successfully and return the variable key!
Start retrieving datasets of experiment 1 ensemble number 6
Retrieving datasets successfully and return the variable key!
Start retrieving datasets of experiment 1 ensemble number 7
Retrieving datasets successfully and return the variable key!
Start retrieving dataset

/home/ESLT0068/NLeSC/Computation_Modeling/BlueAction/WP3/Scripts/JointAnalysis/Analysis/analyzer.py:434: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  a[i,j], b[i,j] = np.linalg.lstsq(A,series[:,i,j])[0]


NameError: name 'meta' is not defined